In [1]:
import os
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import base64
from io import BytesIO
import pandas as pd
import dash_table
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from dash.dash_table.Format import Group
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from pathlib import Path
from dash_table.Format import Format

/var/folders/mg/shf2_k5s0rq_ml6sdq03rj500000gn/T/ipykernel_74934/1776933874.py:9: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

In [3]:
index_page = html.Div(
    style={
        'display': 'flex',
        'flex-direction': 'column',
        'justify-content': 'flex-start',
        'align-items': 'center',
        'height': '100vh',
        'background-image': 'url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ1PVG3eucA4ZSxsD3MD-1RIE-DQlufqxgmkQ&usqp=CAU")',  # Use the image file name
        'background-size': 'cover',
        'background-repeat': 'no-repeat',
        'position': 'relative',  # Ensure the background covers the entire viewport
    },
    children=[
        html.H1("Welcome to Employee Portal", style={'font-family': 'Corbel', 'font-size': '54px', 'color': 'black'}),
        html.Br(),
        html.Div(
            style={'text-align': 'left'},
            children=[
                html.Label('Login', style={'font-family': 'corbel', 'font-size': '24px', 'color': 'black'}),
                dcc.Input(id='login-input', type='text', style={'font-family': 'corbel', 'font-size': '24px', 'margin-left': '42px'})
            ]
        ),
        html.Div(
            style={'text-align': 'left','margin-bottom': '8em'},
            children=[
                html.Label('Password', style={'font-family': 'corbel', 'font-size': '24px', 'color': 'black'}),
                dcc.Input(id='password-input', type='password', style={'font-family': 'corbel', 'font-size': '24px', 'margin-left': '10px'})
            ]
        ),
        html.Button('Submit', id='login-button', n_clicks=0, style={'background-color': 'red', 'color': 'white'}),
        html.Div(id='login-error')
    ]
)

In [4]:
Employee = pd.read_csv('employee.csv')
Employee = Employee.rename(columns=lambda x: x.strip())
Employee_count = Employee['Name'].count()
Assigned_sum = Employee['Assigned_US'].sum()
Open_sum = Employee['Open_US'].sum()
Closed_sum = Employee['Closed_US'].sum()
Team_count = Employee['Teams'].nunique()

In [5]:
def create_hexagon_box(title, count):
    hexagon_box = html.Div(
        style={'position': 'relative', 'width': '200px', 'height': '220px', 'border': '2px dashed black', 'background-color': '#aec5eb',
               'clip-path': 'polygon(50% 0%, 100% 25%, 100% 75%, 50% 100%, 0% 75%, 0% 25%)'},
        children=[
            html.H1(
                style={'position': 'absolute', 'top': '50%', 'left': '50%', 'transform': 'translate(-50%, -50%)',
                       'font-family': 'Arial, sans-serif', 'color': 'black', 'text-align': 'center', 'margin': '0', 'padding': '20px'},
                children=[
                    title,
                    html.Br(),
                    f'{count}'
                ]
            )
        ]
    )
    return hexagon_box


In [6]:
Overview = html.Div(
    style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'flex-wrap': 'wrap'},
    children=[
        html.Div(
            style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'margin': '10px'},
            children=[create_hexagon_box('Employees', Employee_count)]
        ),
        html.Div(
            style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'margin': '10px'},
            children=[create_hexagon_box('Open User Stories', Open_sum)]
        ),
        html.Div(
            style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'margin': '10px'},
            children=[create_hexagon_box('Closed User Stories', Closed_sum)]
        ),

        html.Div(
            style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'margin': '10px'},
            children=[create_hexagon_box('Teams', Team_count)]
        )
    ]
)



In [7]:
 #Employee_Graph
Graph_Employee = html.Div([
    html.Div([
        html.H1('Dashboard'),
        dcc.Dropdown(id='dropdown1', options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': team, 'value': team} for team in Employee['Teams'].unique()], value='ALL'),
        dcc.Dropdown(id='dropdown2', options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': employee, 'value': employee} for employee in Employee['Name'].unique()], value='ALL'),
        dcc.RangeSlider(id='rank-slider', min=1, max=9, value=[1,9],step=1),
    ], style={'width': '25%', 'position': 'fixed', 'overflowY': 'auto', 'height': '100vh'}),

    html.Div([
        dcc.Graph(id='graph1'),
    ], style={'marginLeft': '25%'})
])

# Step 6: Define the callback functions for the interactive components
@app.callback(Output('graph1', 'figure'), [Input('dropdown1', 'value'), Input('dropdown2', 'value')])
def update_graph1(team, employee):
    filtered_df = Employee.copy()

    if team != 'ALL':
        filtered_df = filtered_df[filtered_df['Teams'] == team]
    if employee != 'ALL':
        filtered_df = filtered_df[filtered_df['Name'] == employee]

    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df['Month'] = filtered_df['Date'].dt.month
    grouped_df = filtered_df.groupby(['Month', 'Name'])['Performance'].mean().reset_index()

    fig = go.Figure()

    # Sort the DataFrame by month and name
    grouped_df = grouped_df.sort_values(['Month', 'Name'])

    # Define the order of months
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    for employee_name, employee_df in grouped_df.groupby('Name'):
        x_smooth = pd.to_datetime(employee_df['Month'], format='%m').dt.strftime('%b')
        fig.add_trace(go.Scatter(x=x_smooth, y=employee_df['Performance'], mode='lines+markers', name=employee_name))

    fig.update_layout(
        xaxis=dict(
            title='Month',
            tickmode='array',
            tickvals=[i+1 for i in range(12)],  # 1 to 12 for the months
            ticktext=month_order
        ),
        yaxis_title='Performance Score',
        title='MONTHLY PERFORMANCE SCORE BY EMPLOYEE',
        showlegend=True
    )

    return fig


In [8]:
# user story Graph

Graph_UserStories = html.Div([
    html.Div([
        html.H1('Dashboard'),
        dcc.Dropdown(id='team-dropdown', options=[{'label': team, 'value': team} for team in Employee['Teams'].unique()], placeholder='Select Team'),
        dcc.Dropdown(id='employee-dropdown', placeholder='Select Employee'),
    ], style={'width': '25%', 'position': 'fixed', 'overflowY': 'auto', 'height': '100vh'}),

    html.Div([
        dcc.Graph(id='user-stories-graph'),
    ], style={'marginLeft': '25%'})
])
@app.callback(
    Output('employee-dropdown', 'options'),
    [Input('team-dropdown', 'value')]
)
def update_employee_dropdown(selected_team):
    if selected_team:
        employees = Employee[Employee['Teams'] == selected_team]['Name'].unique()
        options = [{'label': emp, 'value': emp} for emp in employees]
    else:
        options = []
    return options


@app.callback(
    Output('user-stories-graph', 'figure'),
    [Input('team-dropdown', 'value'), Input('employee-dropdown', 'value')]
)
def update_graph(team, employee):
    filtered_df = Employee.copy()

    if team:
        filtered_df = filtered_df[filtered_df['Teams'] == team]
    if employee:
        filtered_df = filtered_df[filtered_df['Name'] == employee]

    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df['Month'] = filtered_df['Date'].dt.strftime('%b')
    grouped_df = filtered_df.groupby('Month').agg({'Assigned_US': 'sum', 'Open_US': 'sum', 'Closed_US': 'sum'}).reset_index()

    fig = go.Figure()

    fig.add_trace(go.Bar(x=grouped_df['Month'], y=grouped_df['Open_US'], name='Open User Stories', marker_color='red'))
    fig.add_trace(go.Bar(x=grouped_df['Month'], y=grouped_df['Closed_US'], name='Closed User Stories', marker_color='green'))
    
    if employee:
        fig.add_trace(go.Bar(x=grouped_df['Month'], y=grouped_df['Assigned_US'], name='Assigned User Stories', marker_color='blue'))

    fig.update_layout(
        xaxis=dict(title='Month'),
        yaxis=dict(title='Count of User Stories'),
        barmode='group',
        title='User Stories Analysis',
        showlegend=True,
        legend=dict(x=1.05, y=1)
    )

    return fig


In [9]:
# Teams story Graph
Graph_Teams = html.Div([
    html.Div([
        html.H1('Dashboard'),
        dcc.Dropdown(id='dropdown', options=[{'label': team, 'value': team} for team in Employee['Teams'].unique()], placeholder='Select Team'),
    ], style={'width': '25%', 'position': 'fixed', 'overflowY': 'auto', 'height': '100vh'}),

    html.Div([
        dcc.Graph(id='user-stories-performance-graph'),
    ], style={'marginLeft': '25%'})
])
# Define the callback function for the interactive component
@app.callback(
    Output('user-stories-performance-graph', 'figure'),
    [Input('dropdown', 'value')]
)
def update_graph(team):
    filtered_df = Employee.copy()

    if team:
        filtered_df = filtered_df[filtered_df['Teams'] == team]

    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df['Month'] = filtered_df['Date'].dt.strftime('%b')
    grouped_df = filtered_df.groupby('Month').agg({'Open_US': 'sum', 'Closed_US': 'sum', 'Performance': 'mean'}).reset_index()

    fig = go.Figure()

    # Bar chart for User Stories
    fig.add_trace(go.Bar(x=grouped_df['Month'], y=grouped_df['Open_US'], name='Open User Stories', marker_color='red'))
    fig.add_trace(go.Bar(x=grouped_df['Month'], y=grouped_df['Closed_US'], name='Closed User Stories', marker_color='green'))

    # Line chart for Performance
    fig.add_trace(go.Scatter(x=grouped_df['Month'], y=grouped_df['Performance'], mode='lines+markers', name='Performance', yaxis='y2', marker_color='blue'))

    fig.update_layout(
        xaxis=dict(title='Month'),
        yaxis=dict(title='Count of User Stories'),
        yaxis2=dict(title='Performance Score', overlaying='y', side='right'),
        title='User Stories and Performance Analysis',
        showlegend=True,
        legend=dict(x=1.05, y=1)
    )

    return fig

In [10]:
Page_layout_2 = html.Div([
    html.Div(
        dcc.Link('Back', href='/', style={'float': 'right', 'margin': '10px'})
    ),
    html.Div([
        dcc.Tabs(id='tabs',value='tab-1',
            children=[
                dcc.Tab(label='Initial Insights',value='tab-1',
                    children=[Overview],  # Add your 'overview' content here
                    selected_style={'backgroundColor': '#E56D00', 'color': 'black','fontWeight': 'bold'},
                    style={'backgroundColor': '#808080', 'zIndex': '1', 'color': 'black','fontWeight': 'bold'}
                ),
                dcc.Tab(label='Employee',value='tab-2',
                    children=[html.H3(Graph_Employee)],
                    selected_style={'backgroundColor': '#E56D00', 'color': 'black','fontWeight': 'bold'},
                    style={'backgroundColor': '#808080', 'zIndex': '1','color': 'black','fontWeight': 'bold'}
                        
                ),
                dcc.Tab(label='User Stories',value='tab-3',
                    children=[html.H3(Graph_UserStories)],
                    selected_style={'backgroundColor': '#E56D00', 'color': 'black','fontWeight': 'bold'},
                    style={'backgroundColor': '#808080', 'zIndex': '1','color': 'black','fontWeight': 'bold'}
                ),
                dcc.Tab(label='Teams',value='tab-4',
                    children=[html.H3(Graph_Teams)],
       selected_style={'backgroundColor': '#E56D00', 'color': 'black','fontWeight': 'bold'},
     style={'backgroundColor': '#808080', 'zIndex': '1','color': 'black','fontWeight': 'bold'}
        ),
    ]),
    html.Div(
        id='tab-content',
        style={'margin-top': '80px', 'height': 'calc(100vh -120px)', 'overflow': 'scroll'}
    )
],
    style={'padding-top': '40px'})

]
                        )


In [11]:
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/page-1':
        return page_layout
    elif pathname == '/page-2':
        return Page_layout_2
    else:
        return index_page

In [12]:
# call back for Validation check

@app.callback(
    Output('login-error', 'children'),
    [Input('login-button', 'n_clicks')],
    [State('login-input', 'value'), State('password-input', 'value')]
)
def validate_login(n_clicks, username, password):
    if n_clicks > 0:
        if (username == 'Admin' and password == 'Nandhini') :
            return dcc.Link('next', href='/page-2')
        else:
            return html.Div('Wrong credentials! Try again.', style={'color': 'red'})

In [13]:
@app.callback(
    Output('url', 'pathname'),
    [Input('next-button', 'n_clicks')]
)
def redirect_to_page_2(n_clicks):
    if n_clicks > 0:
        return '/page-2'
    else:
        raise PreventUpdate

In [14]:
if __name__ == '__main__':
    app.run(host='127.0.0.1',debug=False)

/var/folders/mg/shf2_k5s0rq_ml6sdq03rj500000gn/T/ipykernel_74934/3371463520.py:25: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/mg/shf2_k5s0rq_ml6sdq03rj500000gn/T/ipykernel_74934/2531952481.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/mg/shf2_k5s0rq_ml6sdq03rj500000gn/T/ipykernel_74934/2081305396.py:39: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/mg/shf2_k5s0rq_ml6sdq03rj500000gn/T/ipykernel_74934/2081305396.py:39: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`.